In [116]:
from flask import Flask, render_template_string, request, url_for
import os

app = Flask(__name__)

# Initialize the neutrality score at 0.5
neutrality_score = 0.5

# List of available images in the static folder
folder_path = "static"

# Get a list of all file names in the folder
available_images = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

#start page template
start_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Start Page</title>
    <style>
    .start-button {
         margin-top: 30px;
            padding: 15px 30px;
            font-size: 18px;
            background-color: #E8DEC9;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            }
        @keyframes shrink {
            0% {transform: scale(1.1); }
            100% {transform: scale(0.8); }
        }
    .shrink-image {
        width: 600px;
        height: auto;
        animation: shrink 1s ease-in-out forwards;
    }
    </style>
</head>

<body>
    <h1> Welcome to Rabbit </h1>
     <form action="/home" method="get">
        <button type="submit" class="start-button">Start</button>
    </form>
    <img src="{{ url_for('static', filename='rabbitlogo-removebg.png') }}" alt="Logo" class="shrink-image" >
</body>
</html>

"""


# Main page HTML template with clickable images and image scores
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Flask Button Example</title>
     <style>
        body {
            background-color: #F5EFE0;
            padding-top: 10px;
        }

        .banner {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }

     
        .banner img {
            height: 120%;
            width: auto;
            object-fit: contain;
        }
        

        .image-container {
            display: flex;
            justify-content: center; /* Centers horizontally */
            align-items: center; /* Aligns items in the middle */
            gap: 20px; /* Adds spacing between images */
            margin-top: 200px; /* Adds some space from the top */
        
        }
        .image-container img {
            width: 450px;
            height: auto;
            display: block;
            border-radius: 15px;
        }
        
    </style>
</head>

<body>
    
<div class="banner">
    <img src="{{ url_for('static', filename='rabbitlogo-removebg.png') }}" alt="Logo" >
</div>



        <div class="image-container">
    <form method="POST" action="/button-clicked/{{ images[0].filename }}" style="display: inline;">
        <button type="submit" style="border: none; background: none; padding: 0;">
            <img src="{{ url_for('static', filename=images[0].filename) }}" alt="Click to see {{ images[0].filename }}" >
        </button>
       
    </form>
    
    <form method="POST" action="/button-clicked/{{ images[1].filename }}" style="display: inline;">
        <button type="submit" style="border: none; background: none; padding: 0;">
            <img src="{{ url_for('static', filename=images[1].filename) }}" alt="Click to see {{ images[1].filename }}" >
        </button>
 
    </form>


    </div>
</body>
</html>
"""

# Template to display the clicked image
button_clicked_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Button Clicked</title>
</head>
<body>
    <h1>Button was clicked!</h1>
    <p>Current Neutrality Score: {{ neutrality_score }}</p>
    <img src="{{ url_for('static', filename=image_filename) }}" alt="Clicked Image" style="width:300px;height:auto;">
    
    <p>Image Score: {{ image_score }}</p>  <!-- Display the image score here -->
    
    <form action="/" method="get">
        <button type="submit">Back to Home</button>
    </form>
    <form method="POST" action="/like">
        <input type="hidden" name="image_filename" value="{{ image_filename }}">
        <button type="submit">Like (+0.1)</button>
    </form>
    <form method="POST" action="/dislike">
        <input type="hidden" name="image_filename" value="{{ image_filename }}">
        <button type="submit">Dislike (-0.1)</button>
    </form>
</body>
</html>
"""


@app.route("/", methods=["GET"])
def start():
        return render_template_string(start_template)

@app.route("/home", methods=["GET"])
def home():
    global neutrality_score
    # Randomly select two different images for display
    nr_images = len(available_images)
    
    # Calculate image score based on position
    images = []
    for idx, filename in enumerate(available_images):
        image_score = idx / nr_images  # Image score calculation
        images.append({'filename': filename, 'score': round(image_score, 2)})  # Store both filename and score
    
    # Select two images based on the neutrality_score
    
    if neutrality_score == 1:
        image1 = int(neutrality_score * nr_images)-1
        image2 = image1 - 1
    elif neutrality_score == 0:
        image1 = int(neutrality_score * nr_images)
        image2 = image1 + 1
    else:
        image1 = int(neutrality_score * nr_images)
        image2 = image1 + 1

    selected_images = [images[image1], images[image2]]
    return render_template_string(html_template, neutrality_score=neutrality_score, images=selected_images)

@app.route("/button-clicked/<image_filename>", methods=["POST"])
def button_clicked(image_filename):
    global neutrality_score
    
    # Recalculate the image list (just like in the home route) to get the image score
    nr_images = len(available_images)
    images = []
    for idx, filename in enumerate(available_images):
        image_score = idx / nr_images  # Image score calculation
        images.append({'filename': filename, 'score': round(image_score, 2)})  # Store both filename and score
    
    # Find the image with the corresponding filename
    image = next(image for image in images if image['filename'] == image_filename)
    image_score = image['score']  # Get the image score
    
    return render_template_string(button_clicked_template, neutrality_score=neutrality_score, image_filename=image_filename, image_score=image_score)

@app.route("/like", methods=["POST"])
def like():
    global neutrality_score
    image_filename = request.form.get("image_filename")
    neutrality_score = min(1.0, neutrality_score + 0.1)

    return render_template_string(button_clicked_template, neutrality_score=neutrality_score, image_filename=image_filename)

@app.route("/dislike", methods=["POST"])
def dislike():
    global neutrality_score
    image_filename = request.form.get("image_filename")
    neutrality_score = max(0.0, neutrality_score - 0.1)
    return render_template_string(button_clicked_template, neutrality_score=neutrality_score, image_filename=image_filename)

# Run the Flask app
from werkzeug.serving import run_simple

if __name__ == "__main__":
    run_simple("localhost", 5001, app)

 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [15/Feb/2025 22:22:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2025 22:22:05] "GET /static/rabbitlogo-removebg.png HTTP/1.1" 304 -
127.0.0.1 - - [15/Feb/2025 22:22:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2025 22:22:09] "GET /static/rabbitlogo-removebg.png HTTP/1.1" 304 -
127.0.0.1 - - [15/Feb/2025 22:22:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2025 22:22:11] "GET /static/rabbitlogo-removebg.png HTTP/1.1" 304 -
